In [1]:
import warnings

warnings.filterwarnings("ignore")
import os

if "jbook" in os.getcwd():
    os.chdir(os.path.abspath(os.path.join("../..")))
FORCE = True

# Data Quality Analysis for Aspect-Based Sentiment Analysis with Transformer Models
Data Quality Anomaly Detection (DQAD) and analysis are standard operating procedures in NLP, and for good reason: machine learning models often struggle with performance-degrading inconsistencies and irrelevant artifacts in text data. Whereas, these *traditional* models, highly sensitive to noise, require careful preprocessing to function effectively; *transformer* models such as **GPT-2** {cite}`BetterLanguageModels`, **BERT** {cite}`devlinBERTPretrainingDeep2019a`, and its descendants, **RoBERTA** {cite}`liuRoBERTaRobustlyOptimized2019`, and **DeBERTa** {cite}`heDeBERTaDecodingenhancedBERT2021`, exhibit greater stability and robustness due in part to fine-tuned on **user-generated content** (UGC) datasets and certain architectural differences: **subword tokenization**, **contextual embeddings**, and **self-attention mechanisms**.

**Subword Tokenization**   
Experiments have demonstrated that subword tokenization methods such as Byte-Pair Encoding {cite}`Gage1994ANA`, WordPiece, and SentencePiece {cite}`kudoSentencePieceSimpleLanguage2018` significantly improve transformer model performance in natural language processing tasks such as machine translation and sentence classification. By breaking down words into frequently-occurring subword units, models can more effectively deal with rare and out-of-vocabulary words, leading to better overall accuracy and performance. 

**Contextual Embeddings**    
Given a corpus, static tokenizers such as Word2Vec {cite}`mikolovEfficientEstimationWord2013` are idempotent, fixed, and context-independent. That is, in the sentence "From atop the *bank*, we saw the waves roll in.", the word `bank` will always be assigned the same unique vector, regardless of the context in which it appears. Contextual embeddings, by contrast, are dynamic and context-dependent, allowing models to *embed* the semantic meaning of a word within a sequence.

**Self-Attention**   
Central to transformer-based models' ability to capture context is the **self-attention** mechanism {cite}`chengLongShortTermMemoryNetworks2016`; whereby, each token in a sequence is represented by three vectors:  
- **Key** ($𝑘$k): Encodes what each token offers in terms of information and context.
- **Query** ($𝑞$): Encodes what each token is seeking from others in the sequence.
- **Value** ($𝑣$v): Contains the actual information each token contributes.    

The dot product of the query and key vectors calculates a relevance score, quantifying how much each token should attend to others. These scores are used to weight the value vectors, allowing the model to dynamically aggregate contextual information across the sequence. For example, in the sentence "The cat sat on the mat", the query for "sat" may attend more to "cat" (subject) and "mat" (location) based on their relevance.

**Fine-Tuning on UGC Datasets**   
Finally, fine-tuning on UGC datasets such as the **IMDB Movie Reviews Dataset** {cite}`maas-EtAl:2011:ACL-HLT2011`, **Laptop Reviews**, and **Restaurant Reviews** {cite}`pontiki-etal-2014-semeval`, exposes models to informal and noisy text, aligning them more closely with the characteristics of real-world language.

In all, these facets interact to make transformer models not only robust to real-world noise, grammatical errors, informal word forms, and misspellings, but also allow them to derive semantic meaning from slang, emojis, emoticons, abbreviations, and internet jargon. While traditional preprocessing aims to eliminate such noise, overprocessing this 'useful noise' - grammatical errors, informal expressions, and idiosyncratic features - can hinder, rather than enhance, model performance and generalization {cite}`languageandmultimodalailamalabimperialcollegelondonukBetterUnderstandingNoise2021`. By preserving or even embracing this *useful* noise, we leverage transformer model architectural strengths, enabling them to become more adaptable and generalizable in practical applications.

## Data Quality Anomaly Detection Approach
Next, we detect data quality anomalies in the dataset along four + 1 dimensions of data quality: **validity**, **completeness**, **relevance**, **uniqueness**, and **privacy and compliance**.

### Validity Dimension
Valid data adhere to expected formats, structures, encoding, patterns, values, and ranges.

1. **Values**: 
    - **Rating**: Ensure that `rating` values are discrete values in [1,5]
    - **Category**: All `category` values must be one of the 14 categories evaluated in this study.
    - **Review Date**: Review `date` values must be between January 1, 2020, and September 30, 2023.

2. **Structure**: Evaluate the degree to which the data are represented with appropriate data types, and are free of corrupting noise that carries no inherent meaning for modeling. Noise flagged during the data quality anomaly detection stage are:
    - **Excessive Special Characters**: The presence of excessive or random special characters can corrupt the intended meaning of the text and make it harder for models to interpret context. Accuracy checks ensure that these characters are only present when they add legitimate semantic value, such as in programming-related text or stylized writing.  
    - **Non-ASCII Characters**: While transformers can process non-ASCII characters, they may introduce unintended complexities or errors, especially when non-ASCII content is mixed into primarily English text without a clear purpose. Validity checks flag these occurrences to determine if they are contextually appropriate or represent an error in the data.  
    - **Control Characters**: Control characters, which are non-printable characters like tabs or line breaks embedded in text data, can disrupt text parsing and processing. Ensuring their absence or appropriate use maintains the structural accuracy needed for smooth NLP operations. 
    - **HTML Characters**: Text data sourced from the web may contain HTML tags or character entities that interfere with the text's readability and model understanding. Accuracy checks to sanitize or transform these elements to their intended textual form.  
    - **Excessive Whitespace**: Extra spaces or line breaks, though seemingly minor, can affect text tokenization and representation in models. Normalizing whitespace ensures text is processed in a consistent, meaningful way.  
    - **Accented and Diacritic Characters**: While accented characters are valid in many languages, their unintended presence in primarily non-accented text can indicate data entry errors. Checks for these characters verify if they are linguistically appropriate or require correction. 

3. **Patterns**: Pattern analysis can expose signals of SPAM, non-sensical sequences of text, and excessive repetition. Data quality anomalies of interest include:
    - **Repeated Sequences**: Reviews with excessive repetition of sequences, such as repeated letters, words, or patterns, can indicate spam or low-quality content. Validity checks identify such sequences, enabling corrective measures to ensure high-quality input for NLP models. 
    - **Repeated Words**: Similar to repeated sequences, the presence of redundant words may indicate automated or spam-like content. Detecting and addressing these issues helps maintain the linguistic integrity of the dataset.  
    - **Repeated Phrases**: Repeated phrases can dilute the semantic richness of the text and may signify low-quality or irrelevant content. Validity checks ensure these phrases are flagged for removal or further examination. 
    - **Elongation**: Text elongation, like in "sooo coool," is often used to emphasize words but may not be handled uniformly by models. Accuracy checks flag or normalize elongation to ensure consistent semantic interpretation. 

4. **Information**: In the context of language models, low perplexity is another signal of repetitive or predictable patterns that may not carry substantive meaning. Flagging low perplexity reviews ensures that text has appropriate complexity and variability for analysis and modeling.

By incorporating these **validity** checks, we isolate observations requiring further scrutiny and treatment during the data cleaning stage.

### Completeness Dimension
> Every valid logical expression is provable. Equivalently, every logical expression is either satisfiable or refutable.  (Gödel 1929). 

Here we refer to the Completeness *Dimension*, not to be confused with Gödel’s Completeness *Theorem* above! (*This is what happens we you make computer scientists take philosophy classes*). Primarily, our focus is NULL values in the dataset. A secondary examination is the degree to which the data are complete in terms of representation along two dimensions: **categories** and **sentiments**. Completeness mitigates against model bias and prediction errors derived from class imbalance and under/over-representation.

### Relevance Dimension
The **Relevance** dimension in text data quality ensures that the content is contextually appropriate and meaningful for the specific NLP task or analysis at hand. In other words, the text must be pertinent to the domain, language, or focus of the project. Relevance checks filter out content that could mislead models or degrade the performance of algorithms by introducing off-topic or linguistically inconsistent information.
1. **Non-English App Names**: In datasets where the primary focus is on English-language content, non-English app names can be a source of confusion or skew analysis results. Relevance checks flag these instances, allowing us to either exclude or process them separately to maintain linguistic consistency. 
2. **Non-English Review Text**: Similar to non-English app names, reviews written in languages other than English may be irrelevant to models trained specifically on English text. Relevance checks identify non-English text, helping ensure the data aligns with the model's language capabilities and task requirements. 
3. **Review Length < 3**: Very short reviews, typically less than three words, often lack substantive information or context. These reviews are unlikely to provide meaningful insights and may act as noise, affecting sentiment analysis or topic modeling performance. Relevance checks filter these short reviews to maintain a focus on text that contributes valuable content to the analysis.

By assessing relevancy, we ensure that the text data are appropriate, meaningful, and aligned with the goals of the analysis. This dimension helps avoid the inclusion of extraneous or off-topic content that could distort model training or analysis results.

### Uniqueness Dimension
The **Uniqueness** dimension examines the degree to which duplication is extant. Duplicated content can skew analysis, reduce the diversity of linguistic features, and lead to misleading insights. Therefore, **uniqueness** crucially influences the integrity and reliability of analytical results and model inference. Here, we evaluate the uniqueness of:
1. **Records**: Duplicate records are flagged for analysis and removal during the data cleaning stage.
2. **Review Id**: Duplicate review identifiers can indicate duplication or corruption of the qualitative and quantitative review content. Uniqueness checks detect and flag duplicate review IDs by date, keeping the most recent record is the source of truth. 
3. **Review**: Duplicate reviews are extant when multiple instances exist for the [`app_id`, `author`, `content`] combination.

By enforcing the **Uniqueness** dimension, we ensure that analyses are based on a diverse and representative sample of the text.

### Data Governance: Privacy and Compliance
1. **URLs**: Reviews containing URLs may not provide meaningful textual content for analysis and can disrupt language models. Validity checks identify and flag URLs, allowing for their removal or replacement to maintain textual coherence. 
2. **Phone Numbers**: Similar to URLs, phone numbers are often irrelevant to the semantic content of a review and may interfere with text processing. Validity checks detect phone numbers, ensuring that they are either masked or removed to avoid skewing the analysis. 
3. **Email Addresses**: Email addresses can introduce noise and potentially violate privacy policies. Detecting and handling these elements helps maintain data integrity and privacy while ensuring the text remains analyzable. 

Next, we construct and execute the **Data Quality Anomaly Detection** pipeline, adding indicators of data validity, completeness, relevance, uniqueness, and compliance.


## Data Quality Anomaly Detection Pipeline
Following our standard pipeline pattern, we load the pipeline configuration, then construct and run the DataQualityStage piepline. 

### Import Libraries

In [2]:
from discover.container import DiscoverContainer
from discover.infra.config.flow import FlowConfigReader
from discover.core.flow import StageDef
from discover.flow.stage.data_prep.quality import DataQualityStage
from discover.assets.idgen import AssetIDGen
from discover.core.flow import PhaseDef, StageDef

### Dependency Container

In [3]:
container = DiscoverContainer()
container.init_resources()
container.wire(
    modules=[
        "discover.flow.stage.base",
    ],
)

In [4]:
# Obtain the configuration
reader = FlowConfigReader()
stage_config = reader.get_stage_config(phase=PhaseDef.DATAPREP, stage=StageDef.DQD)
# Build and run the stage
stage = DataQualityStage.build(
    stage_config=stage_config, return_dataset=False, force=FORCE
)
dataset = stage.run()

[11/24/2024 03:50:10 AM] [DEBUG] [discover.flow.stage.base.DataQualityStage] [run] : Execution path: RUN
[11/24/2024 03:50:10 AM] [DEBUG] [discover.infra.service.spark.session.SparkSessionPool] [_create_session] : Creating a spark session.
[11/24/2024 03:50:10 AM] [DEBUG] [discover.infra.service.spark.session.SparkSessionPool] [_create_session] : Creating an Spark session. log4j Configuration: file:/home/john/projects/appvocai-discover/log4j.properties




# ============================================================================== #
#                      Data Quality Anomaly Detection Stage                      #
# ============================================================================== #



your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
[11/24/2024 03:50:24 AM] [DEBUG] [Anomaly.run] [wrapper] : Task: DetectOrRepairURLTask
[11/24/2024 03:50:24 AM] [DEBUG] [Anomaly.run] [wrapper] : Started: Sun, 24 Nov 2024 03:50:24
[11/24/2024 03:50:24 AM] [DEBUG] [Anomaly.run] [wrapper] : Completed: Sun, 24 Nov 2024 03:50:24
[11/24/2024 03:50:24 AM] [DEBUG] [Anomaly.run] [wrapper] : Runtime: 0.15 seconds




                             DetectOrRepairURLTask                              
                             ---------------------                              
                          Start Datetime | Sun, 24 Nov 2024 03:50:24
                       Complete Datetime | Sun, 24 Nov 2024 03:50:24
                                 Runtime | 0.15 seconds


                         DetectOrRepairEmailAddressTask                         
                         ------------------------------                         
                          Start Datetime | Sun, 24 Nov 2024 03:50:24


[11/24/2024 03:50:24 AM] [DEBUG] [Anomaly.run] [wrapper] : Task: DetectOrRepairEmailAddressTask
[11/24/2024 03:50:24 AM] [DEBUG] [Anomaly.run] [wrapper] : Started: Sun, 24 Nov 2024 03:50:24
[11/24/2024 03:50:24 AM] [DEBUG] [Anomaly.run] [wrapper] : Completed: Sun, 24 Nov 2024 03:50:24
[11/24/2024 03:50:24 AM] [DEBUG] [Anomaly.run] [wrapper] : Runtime: 0.05 seconds
[11/24/2024 03:50:24 AM] [DEBUG] [Anomaly.run] [wrapper] : Task: DetectOrRepairPhoneNumberTask
[11/24/2024 03:50:24 AM] [DEBUG] [Anomaly.run] [wrapper] : Started: Sun, 24 Nov 2024 03:50:24
[11/24/2024 03:50:24 AM] [DEBUG] [Anomaly.run] [wrapper] : Completed: Sun, 24 Nov 2024 03:50:24
[11/24/2024 03:50:24 AM] [DEBUG] [Anomaly.run] [wrapper] : Runtime: 0.04 seconds
[11/24/2024 03:50:24 AM] [DEBUG] [Anomaly.run] [wrapper] : Task: DetectOrRepairControlCharsTask
[11/24/2024 03:50:24 AM] [DEBUG] [Anomaly.run] [wrapper] : Started: Sun, 24 Nov 2024 03:50:24
[11/24/2024 03:50:24 AM] [DEBUG] [Anomaly.run] [wrapper] : Completed: Sun, 24

...

                       Complete Datetime | Sun, 24 Nov 2024 03:50:26
                                 Runtime | 0.07 seconds


                      DetectOrRepairReviewDateAnomalyTask                       
                      -----------------------------------                       
                          Start Datetime | Sun, 24 Nov 2024 03:50:26


[11/24/2024 03:50:26 AM] [DEBUG] [Anomaly.run] [wrapper] : Task: DetectOrRepairReviewDateAnomalyTask
[11/24/2024 03:50:26 AM] [DEBUG] [Anomaly.run] [wrapper] : Started: Sun, 24 Nov 2024 03:50:26
[11/24/2024 03:50:26 AM] [DEBUG] [Anomaly.run] [wrapper] : Completed: Sun, 24 Nov 2024 03:50:26
[11/24/2024 03:50:26 AM] [DEBUG] [Anomaly.run] [wrapper] : Runtime: 0.2 seconds


                       Complete Datetime | Sun, 24 Nov 2024 03:50:26
                                 Runtime | 0.2 seconds


[11/24/2024 03:50:29 AM] [INFO] [discover.infra.persistence.repo.dataset.DatasetRepo] [_remove_dataset_file_by_filepath] : Removed dataset file at workspace/test/dataset/01_dataprep/appvocai_discover-01_dataprep-04_dqd-review-dataset.parquet from repository.
[11/24/2024 03:50:29 AM] [INFO] [discover.infra.persistence.repo.dataset.DatasetRepo] [remove] : Removed dataset dataset-test-dataprep-dqd-review from the repository.
[11/24/2024 03:52:01 AM] [DEBUG] [Stage.run] [wrapper] : Stage: Data Quality Anomaly Detection Stage
[11/24/2024 03:52:01 AM] [DEBUG] [Stage.run] [wrapper] : Stage Started: Sun, 24 Nov 2024 03:50:10
[11/24/2024 03:52:01 AM] [DEBUG] [Stage.run] [wrapper] : Stage Completed: Sun, 24 Nov 2024 03:52:01
[11/24/2024 03:52:01 AM] [DEBUG] [Stage.run] [wrapper] : Stage Runtime: 1.0 minutes and 50.87 seconds




                      Data Quality Anomaly Detection Stage                      
                           Stage Started | Sun, 24 Nov 2024 03:50:10
                         Stage Completed | Sun, 24 Nov 2024 03:52:01
                           Stage Runtime | 1.0 minutes and 50.87 seconds


# ============================================================================ #



With **Data Quality Anomaly Detection** complete, we move on to **Data Quality Analysis (DQA)**.